In [ ]:
cd("/home/htc/bzfsikor/code/OptImpSampling.jl/")
using Pkg; Pkg.activate(".")

In [1]:
# In this notebook we analyse the convergence of different SDE solvers for the 
# Girsanov reweighting of the ISOKANN problem with optimal control
using Revise
using OptImpSampling, StochasticDiffEq, DiffEqNoiseProcess, Plots
using OptImpSampling: Doublewell, isokann, optcontrol, GirsanovSDE, girsanovsample, statify

┌ Info: Precompiling OptImpSampling [e32dacf4-0a2b-4f26-bff4-03a75db77042]
└ @ Base loading.jl:1664
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Your operating system has run out of inotify capacity.
│ Check the current value with `cat /proc/sys/fs/inotify/max_user_watches`.
│ Set it to a higher level with, e.g., `echo 65536 | sudo tee -a /proc/sys/fs/inotify/max_user_watches`.
│ This requires having administrative privileges on your machine (or talk to your sysadmin).
│ See https://github.com/timholy/Revise.jl/issues/26 for more information.
└ @ Revise /home/htc/bzfsikor/.julia/packages/Revise/do2nH/src/packagedef.jl:37
┌ Warning: Your operating system has run out of inotify capacity.
│ Check the current value with `cat /proc/sys/fs/inotify/max_user_watches`.
│ Set it to a higher level with, e.g., `echo 65536 | sudo tee -a /proc/sys/fs/inotify/max_user_watches`.
│ This requires having administrative privileges on your machine (or talk to your sysad

In [ ]:
dynamics = Doublewell()
r = isokann(;dynamics);
plot(x->r.model([x])[1])

In [ ]:
using Random
#Random.seed!(0)
sde = OptImpSampling.SDEProblem(dynamics, [0.])        # basic Doublewell SDE
u = optcontrol(statify(r.model), r.S, sde)                # compute the optimal control from the ISOKANN result
cde = GirsanovSDE(sde, u)
s = solve(cde, save_noise=true, alg=SROCK2())  # solve the girsanov SDE and save the noise
W0 = NoiseWrapper(s.W)                            # extract the noise for convergence analysis
plot(s, label=["Xₜ" "Yₜ"])
chis = r.model(s[1,:]')'
ws = @. exp(-s[2,:])
plot!(s.t, [chis chis .* ws], label=["chi(Xₜ)" "Gₜ"])

In [ ]:
#Random.seed!(0)
@time x,w = girsanovsample(cde, 0.)
@show x,w
e = r.model(x)[1] * w

In [ ]:
# Use a Brownian Tree to get reproducible noise for the convergence analysis of different solvers
W = VirtualBrownianTree(0.0, zeros(2); tree_depth = 5, tend=1.1)

prob = NoiseProblem(W, (0.0, 1.0))
@time sol = solve(prob; dt = 1e-6);

In [ ]:
logspace(min, max, n) = reverse(exp.(range(log(min), log(max), n)))
#W = deepcopy(W0)
W = VirtualBrownianTree(0.0, zeros(2); tree_depth=0, tend=1.1)
function plot_e_convergence!(;min=1e-7, max=1e-2, n=10, alg=EM())

    dts = logspace(min, max, n)
    
    e = map(dts) do  dt
        cde = GirsanovSDE(OptImpSampling.SDEProblem(Doublewell(), [0.], noise=W, alg=alg, dt=dt, abstol=dt, reltol=0.), u)
        @show alg, dt
        @time x,w = girsanovsample(cde, 0.)
        e = r.model(x)[1] * w
    end
    plot!(dts, e, xaxis=:log, label=Base.string(typeof(alg))[1:8])
end

plot()
time = @elapsed for alg in [EM(), LambaEM(), ImplicitEM(), SROCK2()]
    plot_e_convergence!(alg=alg)
end
plot!(xlabel="dt/abstol", ylabel="Z(T)")

In [ ]:
savefig("sdeconvb.png")

In [ ]:
plot!()
xaxis!("dt / atol=rtol")

In [ ]:
using OptImpSampling: CompoundSDE

In [ ]:
CompoundSDE(sde, u)

In [ ]:
Revise.retry()